In [5]:
import numpy as np
import sys
import cv2
from sort import Sort
import mediapipe as mp

## 10.1.1 모션벡터와 광류

In [3]:
# Farneback 알고리즘으로 광류 추정
def draw_OpticalFlow(img,flow,step=16):
    for y in range(step//2,frame.shape[0],step):
        for x in range(step//2,frame.shape[1],step):
            dx,dy=flow[y,x].astype(np.int)
            if(dx*dx+dy*dy)>1:
                cv2.line(img,(x,y),(x+dx,y+dy),(0,0,255),2) 
            else:
                cv2.line(img,(x,y),(x+dx,y+dy),(0,255,0),2)            
    
cap=cv2.VideoCapture("./video/boy.mov")	
if not cap.isOpened(): sys.exit('카메라 연결 실패')
    
prev=None

while(1):
    ret,frame=cap.read()	
    if not ret: sys('프레임 획득에 실패하여 루프를 나갑니다.')
    
    if prev is None:	
        prev=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        continue
    
    curr=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    flow=cv2.calcOpticalFlowFarneback(prev,curr,None,0.5,3,15,3,5,1.2,0)
    
    draw_OpticalFlow(frame,flow)
    cv2.imshow('Optical flow',frame)

    prev=curr

    key=cv2.waitKey(1)	
    if key==ord('q'):	
        break 
    
cap.release()			
cv2.destroyAllWindows() 

C:\Users\82104\AppData\Local\Temp\ipykernel_7740\1357648497.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dx,dy=flow[y,x].astype(np.int)


In [4]:
# 희소 광류 추정을 이용한 KLT 추적
cap=cv2.VideoCapture('./video/slow_traffic_small.mp4')

feature_params=dict(maxCorners=100,qualityLevel=0.3,minDistance=7,blockSize=7)
lk_params=dict(winSize=(15,15),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))

color=np.random.randint(0,255,(100,3))

ret,old_frame=cap.read()		
old_gray=cv2.cvtColor(old_frame,cv2.COLOR_BGR2GRAY)
p0=cv2.goodFeaturesToTrack(old_gray,mask=None,**feature_params)

mask=np.zeros_like(old_frame)	 

while(1):
    ret,frame=cap.read()
    if not ret: break

    new_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    p1,match,err=cv2.calcOpticalFlowPyrLK(old_gray,new_gray,p0,None,**lk_params)	
    
    if p1 is not None:		
        good_new=p1[match==1]
        good_old=p0[match==1]
        
    for i in range(len(good_new)): 
        a,b=int(good_new[i][0]),int(good_new[i][1])
        c,d=int(good_old[i][0]),int(good_old[i][1])
        mask=cv2.line(mask,(a,b),(c,d),color[i].tolist(),2)
        frame=cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
        
    img=cv2.add(frame,mask)
    cv2.imshow('LTK tracker',img)
    cv2.waitKey(30)

    old_gray=new_gray.copy()	
    p0=good_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()

### 10.2.4 프로그래밍 실습: SORT로 사람 추적

In [4]:
def construct_yolo_v3():
    f=open('./saved_model/coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]

    model=cv2.dnn.readNet('./saved_model/yolov3.weights','./saved_model/yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model,out_layers,class_names

def yolo_detect(img,yolo_model,out_layers):
    height,width=img.shape[0],img.shape[1]
    test_img=cv2.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)
    
    yolo_model.setInput(test_img)
    output3=yolo_model.forward(out_layers)
    
    box,conf,id=[],[],[]		
    for output in output3:
        for vec85 in output:
            scores=vec85[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:	
                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)
                w,h=int(vec85[2]*width),int(vec85[3]*height)
                x,y=int(centerx-w/2),int(centery-h/2)
                box.append([x,y,x+w,y+h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind=cv2.dnn.NMSBoxes(box,conf,0.5,0.4)
    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]
    return objects

model,out_layers,class_names=construct_yolo_v3()	
colors=np.random.uniform(0,255,size=(100,3))		

sort=Sort()

cap=cv2.VideoCapture("./video/slow_traffic_small.mp4")
if not cap.isOpened(): sys.exit('카메라 연결 실패')

while True:
    ret,frame=cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    res=yolo_detect(frame,model,out_layers)   
    persons=[res[i] for i in range(len(res)) if res[i][5]==0] 

    if len(persons)==0: 
        tracks=sort.update()
    else:
        tracks=sort.update(np.array(persons))
    
    for i in range(len(tracks)):
        x1,y1,x2,y2,track_id=tracks[i].astype(int)
        cv2.rectangle(frame,(x1,y1),(x2,y2),colors[track_id],2)
        cv2.putText(frame,str(track_id),(x1+10,y1+40),cv2.FONT_HERSHEY_PLAIN,3,colors[track_id],2)            
    
    cv2.imshow('Person tracking by SORT',frame)
    
    key=cv2.waitKey(1) 
    if key==ord('q'): break 
    
cap.release()		
cv2.destroyAllWindows()

### 10.3.1 얼굴 검출

In [7]:
# 정지 영상
img=cv2.imread('./imgs/BSDS_376001.jpg')

mp_face_detection=mp.solutions.face_detection
mp_drawing=mp.solutions.drawing_utils

face_detection=mp_face_detection.FaceDetection(model_selection=1,min_detection_confidence=0.5)
res=face_detection.process(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

if not res.detections:
    print('얼굴 검출에 실패했습니다. 다시 시도하세요.')
else:
    for detection in res.detections:
        mp_drawing.draw_detection(img,detection)
    cv2.imshow('Face detection by MediaPipe',img)

cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
# 비디오
mp_face_detection=mp.solutions.face_detection
mp_drawing=mp.solutions.drawing_utils

face_detection=mp_face_detection.FaceDetection(model_selection=1,min_detection_confidence=0.5)

cap=cv2.VideoCapture("./video/face.mp4")

while True:
    ret,frame=cap.read()
    if not ret:
        print('프레임 획득에 실패하여 루프를 나갑니다.')
        break
    
    res=face_detection.process(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    
    if res.detections:
        for detection in res.detections:
            mp_drawing.draw_detection(frame,detection)
            
    cv2.imshow('MediaPipe Face Detection from video',cv2.flip(frame,1))
    if cv2.waitKey(5)==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

프레임 획득에 실패하여 루프를 나갑니다.


In [12]:
# 증강 현실
dice=cv2.imread('./imgs/dice.png',cv2.IMREAD_UNCHANGED)
dice=cv2.resize(dice,dsize=(0,0),fx=0.1,fy=0.1)
w,h=dice.shape[1],dice.shape[0]

mp_face_detection=mp.solutions.face_detection
mp_drawing=mp.solutions.drawing_utils

face_detection=mp_face_detection.FaceDetection(model_selection=1,min_detection_confidence=0.5)

cap=cv2.VideoCapture("./video/face.mp4")

while True:
    ret,frame=cap.read()
    if not ret:
        print('프레임 획득에 실패하여 루프를 나갑니다.')
        break
    
    res=face_detection.process(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    
    if res.detections:
        for det in res.detections:
            p=mp_face_detection.get_key_point(det,mp_face_detection.FaceKeyPoint.RIGHT_EYE)
            x1,x2=int(p.x*frame.shape[1]-w//2),int(p.x*frame.shape[1]+w//2)
            y1,y2=int(p.y*frame.shape[0]-h//2),int(p.y*frame.shape[0]+h//2)
            if x1>0 and y1>0 and x2<frame.shape[1] and y2<frame.shape[0]:
                alpha=dice[:,:,3:]/255
                frame[y1:y2,x1:x2]=frame[y1:y2,x1:x2]*(1-alpha)+dice[:,:,:3]*alpha
            
    cv2.imshow('MediaPipe Face AR',cv2.flip(frame,1))
    if cv2.waitKey(5)==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 10.3.2 얼굴 그물망 검출

In [16]:
mp_mesh=mp.solutions.face_mesh
mp_drawing=mp.solutions.drawing_utils
mp_styles=mp.solutions.drawing_styles

mesh=mp_mesh.FaceMesh(max_num_faces=2,refine_landmarks=True,min_detection_confidence=0.1,min_tracking_confidence=0.1)

cap=cv2.VideoCapture("./video/face2.mp4")

while True:
    ret,frame=cap.read()
    if not ret:
        print('프레임 획득에 실패하여 루프를 나갑니다.')
        break
    
    res=mesh.process(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    
    if res.multi_face_landmarks:
        for landmarks in res.multi_face_landmarks:
            mp_drawing.draw_landmarks(image=frame,landmark_list=landmarks,connections=mp_mesh.FACEMESH_TESSELATION,landmark_drawing_spec=None,connection_drawing_spec=mp_styles.get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(image=frame,landmark_list=landmarks,connections=mp_mesh.FACEMESH_CONTOURS,landmark_drawing_spec=None,connection_drawing_spec=mp_styles.get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(image=frame,landmark_list=landmarks,connections=mp_mesh.FACEMESH_IRISES,landmark_drawing_spec=None,connection_drawing_spec=mp_styles.get_default_face_mesh_iris_connections_style())
        
    cv2.imshow('MediaPipe Face Mesh',cv2.flip(frame,1))
    if cv2.waitKey(5)==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 10.3.3 손 랜드마드 검출

In [18]:
mp_hand=mp.solutions.hands
mp_drawing=mp.solutions.drawing_utils
mp_styles=mp.solutions.drawing_styles

hand=mp_hand.Hands(max_num_hands=2,static_image_mode=False,min_detection_confidence=0.5,min_tracking_confidence=0.5)

cap=cv2.VideoCapture("./video/hand.mp4")

while True:
    ret,frame=cap.read()
    if not ret:
        print('프레임 획득에 실패하여 루프를 나갑니다.')
        break
    
    res=hand.process(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    
    if res.multi_hand_landmarks:
        for landmarks in res.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame,landmarks,mp_hand.HAND_CONNECTIONS,mp_styles.get_default_hand_landmarks_style(),mp_styles.get_default_hand_connections_style())

    cv2.imshow('MediaPipe Hands',cv2.flip(frame,1))
    if cv2.waitKey(5)==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 10.4.2 BlazePose를 이용한 자세 추정

In [19]:
mp_pose=mp.solutions.pose
mp_drawing=mp.solutions.drawing_utils
mp_styles=mp.solutions.drawing_styles

pose=mp_pose.Pose(static_image_mode=False,enable_segmentation=True,min_detection_confidence=0.5,min_tracking_confidence=0.5)

cap=cv2.VideoCapture("./video/face2.mp4")

while True:
    ret,frame=cap.read()
    if not ret:
        print('프레임 획득에 실패하여 루프를 나갑니다.')
        break
    
    res=pose.process(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    
    mp_drawing.draw_landmarks(frame,res.pose_landmarks,mp_pose.POSE_CONNECTIONS,landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style())
    
    cv2.imshow('MediaPipe pose',cv2.flip(frame,1))
    if cv2.waitKey(5)==ord('q'):
        mp_drawing.plot_landmarks(res.pose_world_landmarks,mp_pose.POSE_CONNECTIONS)
        break

cap.release()
cv2.destroyAllWindows()